In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from datetime import datetime,timedelta  
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

#add
import gc
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import hstack, vstack
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
# from skopt.space import Integer, Categorical, Real, Log10
# from skopt.utils import use_named_args
# from skopt import gp_minimize
from gensim.models import Word2Vec, FastText
import gensim 
import re
import os
path="./"
os.listdir(path)

['xgb_final_nb.csv',
 'deviceid_train.tsv',
 'feat.csv.zip',
 '.DS_Store',
 'thluo_train_best_feat.csv',
 'feat.csv',
 'xgb_feat_final_nb.csv',
 'xgb_nb.ipynb',
 'nurbs_feature_all.csv',
 '.ipynb_checkpoints',
 'deviceid_test.tsv']

In [2]:
train_id=pd.read_csv("deviceid_train.tsv",sep="\t",names=['device_id','sex','age'])
test_id=pd.read_csv("deviceid_test.tsv",sep="\t",names=['device_id'])
all_id=pd.concat([train_id[['device_id']],test_id[['device_id']]])
nurbs=pd.read_csv("nurbs_feature_all.csv")
nurbs.columns=["nurbs_"+str(i) for i in nurbs.columns]
all_id.index=range(len(all_id))
nurbs['device_id']=all_id

In [2]:
th=pd.read_csv("thluo_train_best_feat.csv")

In [4]:
feat=pd.merge(th,nurbs,on="device_id",how="left")

In [5]:
feat.to_csv("feat.csv",index=False)

In [27]:
train=pd.merge(train_id,feat,on="device_id",how="left")
test=pd.merge(test_id,feat,on="device_id",how="left")

In [29]:
features = [x for x in train.columns if x not in ['device_id', 'sex',"age",]]
Y = train['sex'] - 1

In [32]:

import xgboost as xgb
from sklearn.metrics import auc, log_loss, roc_auc_score,f1_score,recall_score,precision_score
from sklearn.cross_validation import StratifiedKFold

kf = StratifiedKFold(Y, n_folds=5, shuffle=True, random_state=1024)
params={
	'booster':'gbtree',
	'objective': 'binary:logistic',
#      'is_unbalance':'True',
# 	'scale_pos_weight': 1500.0/13458.0,
        'eval_metric': "logloss",
    
	'gamma':0.2,#0.2 is ok
	'max_depth':6,
# 	'lambda':20,
    # "alpha":5,
        'subsample':0.7,
        'colsample_bytree':0.4 ,
#         'min_child_weight':2.5, 
        'eta': 0.01,
    # 'learning_rate':0.01,
    "silent":1,
	'seed':1024,
	'nthread':12,
   
    }
num_round = 3500
early_stopping_rounds = 100

/Users/chizhu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [33]:
aus = []
sub1 = np.zeros((len(test), ))
pred_oob1=np.zeros((len(train),))
for i,(train_index,test_index) in enumerate(kf):
  
    tr_x = train[features].reindex(index=train_index, copy=False)
    tr_y = Y[train_index]
    te_x = train[features].reindex(index=test_index, copy=False)
    te_y = Y[test_index]

    # tr_y=tr_y.apply(lambda x:1 if x>0 else 0)
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    d_tr = xgb.DMatrix(tr_x, label=tr_y)
    d_te = xgb.DMatrix(te_x, label=te_y)
    watchlist  = [(d_tr,'train'),
    (d_te,'val')
             ]
    model = xgb.train(params, d_tr, num_boost_round=5500, 
                      evals=watchlist,verbose_eval=200,
                              early_stopping_rounds=100)
    pred = model.predict(d_te)
    pred_oob1[test_index] =pred
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    a = log_loss(te_y, pred)

    sub1 += model.predict(xgb.DMatrix(test[features]))/5
    

    print ("idx: ", i) 
    print (" loss: %.5f" % a)
#     print " gini: %.5f" % g
    aus.append(a)

print ("mean")
print ("auc:       %s" % (sum(aus) / 5.0))

[0]	train-logloss:0.691359	val-logloss:0.691488
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.566693	val-logloss:0.595722
[400]	train-logloss:0.53806	val-logloss:0.590461
[600]	train-logloss:0.519054	val-logloss:0.590032
Stopping. Best iteration:
[529]	train-logloss:0.525748	val-logloss:0.589953

idx:  0
 loss: 0.59015
[0]	train-logloss:0.691215	val-logloss:0.691369
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[200]	train-logloss:0.56648	val-logloss:0.596397
[400]	train-logloss:0.538516	val-logloss:0.591125
[600]	train-logloss:0.51823	val-logloss:0.590809
Stopping. Best iteration:
[595]	train-logloss:0.518718	val-logloss:0.590732

idx:  1
 loss: 0.59099
[0]	train-logloss:0.691228	val-logloss:0.69143
Multiple eval metrics have been passed: 'val-logloss' will be us

In [41]:
pred_oob1 = pd.DataFrame(pred_oob1, columns=['sex2'])
sub1 = pd.DataFrame(sub1, columns=['sex2'])
res1=pd.concat([pred_oob1,sub1])
res1['sex1'] = 1-res1['sex2']

In [48]:
import gc
gc.collect()

1012

In [49]:
####sex1
test['sex']=1

In [50]:
features = [x for x in train.columns if x not in ['device_id',"age"]]
Y = train['age'] 

In [51]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import auc, log_loss, roc_auc_score,f1_score,recall_score,precision_score
from sklearn.cross_validation import StratifiedKFold

kf = StratifiedKFold(Y, n_folds=5, shuffle=True, random_state=1024)
params={
	'booster':'gbtree',
	'objective': 'multi:softprob',
#      'is_unbalance':'True',
# 	'scale_pos_weight': 1500.0/13458.0,
        'eval_metric': "mlogloss",
    'num_class':11,
	'gamma':0.1,#0.2 is ok
	'max_depth':6,
# 	'lambda':20,
    # "alpha":5,
        'subsample':0.7,
        'colsample_bytree':0.4 ,
        # 'min_child_weight':2.5, 
        'eta': 0.01,
    # 'learning_rate':0.01,
    "silent":1,
	'seed':1024,
	'nthread':12,
   
    }
num_round = 3500
early_stopping_rounds = 100

In [52]:
aus = []
sub2 = np.zeros((len(test),11 ))
pred_oob2=np.zeros((len(train),11))
models=[]
iters=[]
for i,(train_index,test_index) in enumerate(kf):
  
    tr_x = train[features].reindex(index=train_index, copy=False)
    tr_y = Y[train_index]
    te_x = train[features].reindex(index=test_index, copy=False)
    te_y = Y[test_index]

    # tr_y=tr_y.apply(lambda x:1 if x>0 else 0)
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    d_tr = xgb.DMatrix(tr_x, label=tr_y)
    d_te = xgb.DMatrix(te_x, label=te_y)
    watchlist  = [(d_tr,'train'),
    (d_te,'val')
             ]
    model = xgb.train(params, d_tr, num_boost_round=5500, 
                      evals=watchlist,verbose_eval=200,
                              early_stopping_rounds=100)
    models.append(model)
    iters.append(model.best_iteration)
    pred = model.predict(d_te,ntree_limit=model.best_iteration)
    pred_oob2[test_index] =pred
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    a = log_loss(te_y, pred)

    sub2 += model.predict(xgb.DMatrix(test[features]),ntree_limit=model.best_iteration)/5
    

    print ("idx: ", i) 
    print (" loss: %.5f" % a)
#     print " gini: %.5f" % g
    aus.append(a)

print ("mean")
print ("auc:       %s" % (sum(aus) / 5.0))

[0]	train-mlogloss:2.39131	val-mlogloss:2.39264
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.80941	val-mlogloss:2.00508
[400]	train-mlogloss:1.60383	val-mlogloss:1.94
[600]	train-mlogloss:1.472	val-mlogloss:1.9241
[800]	train-mlogloss:1.36689	val-mlogloss:1.92024
[1000]	train-mlogloss:1.273	val-mlogloss:1.91999
Stopping. Best iteration:
[918]	train-mlogloss:1.31045	val-mlogloss:1.91983

idx:  0
 loss: 1.91985
[0]	train-mlogloss:2.39114	val-mlogloss:2.39277
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 100 rounds.
[200]	train-mlogloss:1.8078	val-mlogloss:2.0115
[400]	train-mlogloss:1.60116	val-mlogloss:1.9457
[600]	train-mlogloss:1.46953	val-mlogloss:1.93011
[800]	train-mlogloss:1.36553	val-mlogloss:1.92647
Stopping. Best iteration:
[825]	train-mlogloss:1.35318	val-mlogl

In [53]:
res2_1=np.vstack((pred_oob2,sub2))
res2_1 = pd.DataFrame(res2_1)

In [54]:
###sex2
test['sex']=2
features = [x for x in train.columns if x not in ['device_id',"age","label","app"]]
Y = train['age'] 

In [55]:
aus = []
sub2 = np.zeros((len(test),11 ))
for model,it in zip(models,iters):
    sub2 += model.predict(xgb.DMatrix(test[features]),ntree_limit=it)/5
res2_2=np.vstack((pred_oob2,sub2))
res2_2 = pd.DataFrame(res2_2) 

In [56]:
res1.index=range(len(res1))
res2_1.index=range(len(res2_1))
res2_2.index=range(len(res2_2))
final_1=res2_1.copy()
final_2=res2_2.copy()
for i in range(11):
    final_1[i]=res1['sex1']*res2_1[i]
    final_2[i]=res1['sex2']*res2_2[i]
id_list=pd.concat([train[['device_id']],test[['device_id']]])
final=id_list
final.index=range(len(final))
final.columns= ['DeviceID']
final_pred = pd.concat([final_1,final_2],1)
final=pd.concat([final,final_pred],1)
final.columns = ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', 
         '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', 
         '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']

final.to_csv('xgb_feat_final_nb.csv', index=False)

In [57]:
test['DeviceID']=test['device_id']
sub=pd.merge(test[['DeviceID']],final,on="DeviceID",how="left")
sub.to_csv("xgb_final_nb.csv",index=False)